In [161]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pandas_profiling import ProfileReport
import pickle
import os
import datetime
import calendar
from scipy.stats import zscore
import missingno as msno

In [162]:
df_weather = pd.read_csv('IC_Weather_Dat.csv')
df_weather['date_time'] = pd.to_datetime(df_weather['date_time'])
df_weather.drop(df_weather[df_weather['date_time'].dt.year.isin([2023, 2014, 2015])].index, axis=0, inplace=True)
df_precip = df_weather[['date_time', 'wetstate_avg', 'precip_tb', 'precip_wpg', 'precip_7day']]

In [163]:
df_ic_routes, _, _ = pickle.load(open('C:/Users/Brayden/Desktop/LocScrapes/Indian Creek (Full).pkl', 'rb'))
df_ic_routes.drop_duplicates(subset='URL', inplace=True)
df_ic_routes['Base Location'] = df_ic_routes['Base Location'].apply(lambda x: x.strip())

for index, row in df_ic_routes.iterrows():
    row['Route Ticks']['Route'] = row['Route']
    row['Route Ticks']['Base Location'] = row['Base Location']
    row['Route Ticks']['Rating'] = row['Rating']

In [164]:
df_super_tick_list = pd.concat(df_ic_routes['Route Ticks'].to_list())
df_super_tick_list.loc[df_super_tick_list['Username']=='', 'Username'] = 'Private'

In [165]:
(df_precip.isnull().sum()/df_precip.shape[0])*100

date_time        0.000000
wetstate_avg    49.252557
precip_tb        0.000000
precip_wpg      14.358773
precip_7day      0.000000
dtype: float64

In [166]:
px.line(data_frame=df_precip, x='date_time', y=['precip_tb'])

In [167]:
px.line(data_frame=df_precip, x='date_time', y=['wetstate_avg'])
# it would be nice to use this data since it represents present state of dryness but it is too incomplete.

In [169]:
px.histogram(df_precip['precip_tb'].drop(df_precip[df_precip['precip_tb']==0].index, axis=0), marginal='box')

In [73]:
yearly_precip = df_precip.groupby(df_precip['date_time'].dt.year)['precip_tb'].sum()
px.bar(yearly_precip)

In [74]:
df_monthly_precip = df_precip.groupby([df_precip['date_time'].dt.year, df_precip['date_time'].dt.month])['precip_tb'].sum().unstack(level=1)
monthly_precip_mean = df_monthly_precip.mean(axis=0)
px.bar(monthly_precip_mean, title='Average Monthly Sum Rainfall Over 2016-2022 (in)')
# This is sum of rainfall by month, then averaged over the years.
# March April May is very rainy. October less so but November is actually very good.

In [75]:
df_precip_value_counts = df_precip.groupby(df_precip['date_time'].dt.month)['precip_tb'].value_counts().unstack()
df_precip_noprecip_rate = df_precip_value_counts[0.00] / df_precip_value_counts.sum(axis=1)
px.bar(df_precip_noprecip_rate, title='Percentage of Days without Rain')

In [177]:
# There needs to be a rule of thumb for which days are considered violations

enforce_level = 'loose'

# Loosest
# Just days that have serious rainfall
if enforce_level=='loosest':
    wet_days = df_precip[(df_precip['precip_tb']>0.3)]['date_time'].dt.date
    
# Loose
# Just days that have more than a drizzle
if enforce_level=='loose':
    wet_days = df_precip[(df_precip['precip_tb']>0.15)]['date_time'].dt.date

# Strict
# <0.1in : that day. 0.1->0.5 : that day and next. 0.5-1 : that day next two. >1 : that day and next three.
if enforce_level=='heavy':
    drizzle_days = df_precip[(df_precip['precip_tb']>0) & (df_precip['precip_tb']<=0.1)]['date_time'].dt.date

    light_days = df_precip[(df_precip['precip_tb']>0.1) & (df_precip['precip_tb']<=0.5)]['date_time'].dt.date
    light_days = pd.concat([light_days, light_days.apply(lambda x: x+pd.Timedelta(1, 'day'))])

    mediate_days = df_precip[(df_precip['precip_tb']>0.5) & (df_precip['precip_tb']<=1)]['date_time'].dt.date
    mediate_days = pd.concat([mediate_days, mediate_days.apply(lambda x: x+pd.Timedelta(1, 'day')), mediate_days.apply(lambda x: x+pd.Timedelta(2, 'day'))])

    heavy_days = df_precip[(df_precip['precip_tb']>1)]['date_time'].dt.date
    heavy_days = pd.concat([heavy_days, heavy_days.apply(lambda x: x+pd.Timedelta(1, 'day')), heavy_days.apply(lambda x: x+pd.Timedelta(2, 'day')), heavy_days.apply(lambda x: x+pd.Timedelta(3, 'day'))])

    wet_days = pd.concat([drizzle_days, light_days, mediate_days, heavy_days])
    wet_days.drop_duplicates(inplace=True)
    wet_days.sort_values(ascending=True, inplace=True)

wet_days

610     2016-01-09
690     2016-03-29
720     2016-04-28
721     2016-04-29
748     2016-05-26
           ...    
3075    2022-10-23
3076    2022-10-24
3087    2022-11-04
3120    2022-12-07
3142    2022-12-29
Name: date_time, Length: 99, dtype: object

In [178]:
df_tick_infr = df_super_tick_list[df_super_tick_list['Date'].dt.date.isin(wet_days.to_list())]
df_tick_infr

,Username,User Link,Date,Pitches Ticked,Style,Lead Style,Comment,Route,Base Location,Rating
19,Brenna Freer,/user/201102828,2022-04-22,1.0,TR,,First climb in the creek!,Ansaid Tower,Donnelly Canyon,5.7
20,Alex Rose,/user/200664328,2022-04-18,1.0,Lead,Onsight,,Ansaid Tower,Donnelly Canyon,5.7
137,Aj Baeseman,/user/200248588,2019-03-21,1.0,Lead,Onsight,,Ansaid Tower,Donnelly Canyon,5.7
176,Mercadi Carlson,/user/110829158,2016-09-23,1.0,TR,,,Ansaid Tower,Donnelly Canyon,5.7
75,Cam S,/user/200322716,2022-10-23,1.0,TR,,,Binou's Crack,Donnelly Canyon,5.8+
...,...,...,...,...,...,...,...,...,...,...
16,Eben Freeman,/user/107026865,2021-03-27,1.0,,,os,Dreams of Electric Aardvarks,Trick or Treat Wall,5.12-
17,Christine Spang,/user/107323239,2021-03-27,1.0,TR,,Fell once. Hard baggy fingers!,Dreams of Electric Aardvarks,Trick or Treat Wall,5.12-
12,Cierra Heiser,/user/200574086,2021-01-25,1.0,TR,,,Stagefright,Trick or Treat Wall,5.12b
0,TradPrincess Eden,/user/110660719,2022-04-19,1.0,Lead,Redpoint,fell at the top my first go. Came back and sen...,Tricks Are For Kids,Tricks Wall,5.13a/b


In [179]:
df_tick_infr.groupby(df_tick_infr['Date'].dt.year)['Username'].count()

Date
2016    175
2017    184
2018    365
2019    279
2020    303
2021    246
2022    551
Name: Username, dtype: int64

In [180]:
df_tick_infr.groupby(df_tick_infr['Date'].dt.month)['Username'].count()

Date
1      48
2       5
3     515
4     591
5     193
6       2
7       8
8      19
9      68
10    493
11    127
12     34
Name: Username, dtype: int64

In [181]:
df_tick_infr.groupby('Base Location')['Username'].count().sort_values(ascending=False)

Base Location
Supercrack Buttress             303
Donnelly Canyon                 227
Battle of the Bulge Buttress    193
Scarface                        144
Way Rambo                       133
The Optimator                   123
Sparks Wall                      82
2nd Meat Wall                    67
Pistol Whipped                   66
Reservoir Wall                   60
Sinbad Wall aka Nuclear Wall     57
Cat Wall                         56
4X4                              51
Blue Gramma Cliff                50
Selfish Wall                     49
Technicolor Wall                 46
Cliffs of Insanity               45
Critic's Choice                  27
Habitado                         27
Trick or Treat Wall              26
The Wall                         25
South Sixshooter                 25
Broken Tooth                     22
The Beach                        21
The Gash                         18
Petrified Hornet Wall            18
Fin Wall                         18
High Horse    

In [182]:
df_tick_infr.groupby('Route')['Username'].count().sort_values(ascending=False)

Route
Supercrack of the Desert    57
Incredible Hand Crack       53
Generic Crack               52
Chocolate Corner            43
Scarface                    39
                            ..
Double Trouble               1
Six Fingerd Man              1
Skinny Dip                   1
Divided Lines                1
Mr. Critical                 1
Name: Username, Length: 491, dtype: int64

In [183]:
df_tick_infr.groupby('Username')['Route'].count().sort_values(ascending=False)[0:30]

Username
Private              118
Royal                 45
Cierra Heiser         41
Mercadi Carlson       24
Emily Anding          19
Erik Griffith         18
Megan A.              15
Gus Meyers            15
Nathan Woeber         14
chris vanitas         13
Alyssa K              12
Cam S                 12
Geoff Kennard         12
Caleb Carrington      11
Evan Bosso            11
Miles Watson          11
Mallory S             10
Noah Havlik           10
Jacob Wolniewicz      10
Marissa Aguirre       10
Bug Boy               10
Rob Pelon             10
V S                    9
Kirk Woerner           9
Grug M                 9
Kelsey Wagner          9
Calen                  9
Andrew Kelleher        9
Matt Beyer             8
Margalit Goldberg      8
Name: Route, dtype: int64